In [1]:
%pip install -qU langchain
%pip install -qU huggingface_hub
%pip install nest-asyncio
%pip install unstructured
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [18]:
import os
# Necessary to import OpenAI
with open('assets/openai_api_key', 'r') as f:
    openai_api_key = f.read()
with open('assets/huggingface_api_key', 'r') as f:
    huggingface_api_key = f.read()
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['HUGGINGFACEHUB_API_KEY'] = huggingface_api_key
from pathlib import Path
import nest_asyncio
nest_asyncio.apply()
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader
from langchain.output_parsers import RegexParser
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
import json
import pandas as pd
import re

# OpenAI QA

We use Langchain to create a Chroma database of our papers against which we will query our questions. The answers are provided in the requested structure, making it straightforward to extract information.

Every paper is evaluated separately.

TODO: improve documentation

https://python.langchain.com/docs/modules/chains/additional/question_answering.html

https://python.langchain.com/docs/modules/chains/additional/openai_functions_retrieval_qa


Setting up the parameters:

In [3]:
paper_path = '../data/docs/'
pickle_path = '../data/pickle/'
chroma_path = '../data/chroma/'
MODEL = 'gpt-3.5-turbo'

Creating the vector database:

In [4]:
def papers_to_vectors(paper_path = paper_path, chroma_path=chroma_path):
    #TODO chroma_index = chroma_path + 'index'
    loader = DirectoryLoader(paper_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    embeddings = OpenAIEmbeddings()
    docsearch = Chroma.from_documents(texts, embeddings,persist_directory='../data/chroma', metadatas=[{"source": str(i)} for i in range(len(texts))])
    docsearch.persist()
    return docsearch

docsearch = papers_to_vectors()



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Defining the prompt template:

In [5]:
prompt_template = """Use the journal article as context to fill out the keys to return as defined at the end. You will provide an answer with the format below for the source document I have provided. If you don't know the answer, reply with a blank string (''), do not try to make up an answer. I provide an explanation between brackets of what each key to return is, but do not include those in your answer, just the key name exactly as provided followed by a colon and your answer. 

Some examples of adverse outcomes you should look for are: apoptosis, ROS formation or oxidative stress, cell membrane disruption, cell viability reduction, mortality, cell viability, DNA damage, cytotoxicity, pathway activation, accumulation...

In addition to answering to the specific keys, also return a 'Score' of how fully it answered the question, the journal article used as context, and the text passage containing the answer. All these keys have to be in the following #format, do not alter the order or skip any key:

Format:
---------
Question: [question here]
Adverse outcome [adverse outcome here]
Bioassay: [bioassay here]
Material diameter measurement: [diameter measurement here]
Numerical value: [mnumerical value of diameter measurement here]
Units: [units here]
Graphene oxide dose: [dose here]
Graphene oxide dose units: [dose units here]
Organism/Biological system: [test subject here]
Score: [score between 0 and 100]
Source: [source, the journal article containing the answer, from documents metadata]
Passage: [the text passage containing the answer]


---------

Context:
---------
{context}
---------
Questions: 
{question}

"""

output_parser = RegexParser(
    regex=r"\nBioassay: (.*)\nMaterial diameter measurement: (.*)\nNumerical value: (.*)\nUnits: (.*)\nGraphene oxide dose: (.*)\nGraphene oxide dose units: (.*)\nOrganism/Biological system: (.*)\nScore: (.*)\nSource: (.*)\nPassage: (.*)",
    output_keys=["Adverse outcome","Bioassay", "Material diameter measurement", "Numerical value", "Units", "Graphene oxide dose", "Graphene oxide dose units", "Organism/Biological system", "score", "Source", "Passage"],
)

doc_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['context', 'question'],
    output_parser=output_parser,
)


In [17]:
documents = os.listdir(paper_path)
print("- "+ "\n- ".join(documents))


- j.toxlet.2010.11.016.xml
- C2JM31396K.pdf
- nn101097v.pdf
- j.biomaterials.2013.01.001.xml
- acsomega.2c03171.xml
- tx400385x.pdf
- j.biomaterials.2012.07.040.xml
- j.biomaterials.2012.05.064.xml
- smll.201201546.pdf
- nn202699t.pdf
- nn1007176.pdf
- A20060216.pdf
- s11671-010-9751-6.xml
- j.envpol.2017.12.034.xml
- cdd.2010.11.pdf
- nl202515a.pdf
- smll.201102743.pdf
- j.biomaterials.2012.02.021.xml
- j.biomaterials.2012.11.001.xml
- la203607w.pdf
- am300253c.pdf
- ijms221910578.xml


## Diameter

Extracting adverse outcomes related to diameter from each paper:

In [7]:
diameter = """
    Adverse outcome (What was the adverse outcome?):
    Bioassay (What specific bioassay was performed?):
    Material diameter measurement (What type of material diameter measurement is    reported?):
    Numerical value (What is the numerical value of the diameter measurement?):
    Units (What are the units of the diameter measurement, if applicable?):
    Graphene oxide dose (What was the graphene oxide dose that produced this  outcome?):
    Graphene oxide dose units (What are the units of the graphene oxide dose that produced this  outcome?):
    Organism/Biological system (What organism or biological system was the assay performed on (include strain, if applicable)?"""

In [8]:
results = {}
for document in documents:
    query = f"Select the document with name: {document}"
    docs = docsearch.as_retriever().get_relevant_documents(query)
    chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff",    prompt=doc_prompt)
    
    res = chain({"input_documents": docs, "question": diameter},    return_only_outputs=False)
    output = res['output_text'].split("\n")
    results[document] = output
    
output_keys=["Adverse outcome","Bioassay", "Material diameter measurement", "Numerical value", "Units", "Graphene oxide dose", "Graphene oxide dose units", "Organism/Biological system", "score", "Source", "Passage"]
diameter = {key:{} for key in results}
for key in results:
    for output in output_keys:
        for value in results[key]:
            if output in value:
                new_val = value.split(output)[1]
                if ':' in new_val:
                    new_val = new_val.replace(":", "")
                    diameter[key][output] = new_val
df_diameter = pd.DataFrame(diameter).drop('Source') # Not needed

df_diameter.to_csv('../data/results/graphene-oxide/diameter.csv')
df_diameter


,j.toxlet.2010.11.016.xml,C2JM31396K.pdf,nn101097v.pdf,j.biomaterials.2013.01.001.xml,acsomega.2c03171.xml,tx400385x.pdf,j.biomaterials.2012.07.040.xml,j.biomaterials.2012.05.064.xml,smll.201201546.pdf,nn202699t.pdf,...,s11671-010-9751-6.xml,j.envpol.2017.12.034.xml,cdd.2010.11.pdf,nl202515a.pdf,smll.201102743.pdf,j.biomaterials.2012.02.021.xml,j.biomaterials.2012.11.001.xml,la203607w.pdf,am300253c.pdf,ijms221910578.xml
Adverse outcome,Apoptosis,(What was the adverse outcome?) Cell viabilit...,(What was the adverse outcome?) Cell viabilit...,(What was the adverse outcome?) Cell viabilit...,(What was the adverse outcome?) apoptosis,(What was the adverse outcome?) Cell viabilit...,Cell viability reduction,Apoptosis,(What was the adverse outcome?) apoptosis,(What was the adverse outcome?) DNA damage,...,(What was the adverse outcome?) apoptosis,apoptosis,(What was the adverse outcome?) Cell viabilit...,apoptosis,(What was the adverse outcome?) apoptosis,Cell viability reduction,Apoptosis,(What was the adverse outcome?) Cell viabilit...,Cell viability reduction,(What was the adverse outcome?) apoptosis
Bioassay,MTT assay,(What specific bioassay was performed?) MTT a...,(What specific bioassay was performed?) MTT a...,(What specific bioassay was performed?) MTT a...,(What specific bioassay was performed?) MTT a...,(What specific bioassay was performed?) MTT a...,MTT assay,Flow cytometry,(What specific bioassay was performed?) Flow ...,(What specific bioassay was performed?) Comet...,...,(What specific bioassay was performed?) MTT a...,MTT assay,(What specific bioassay was performed?) MTT a...,MTT assay,(What specific bioassay was performed?) Flow ...,MTT assay,Flow cytometry,(What specific bioassay was performed?) MTT a...,Cytotoxicity assay,(What specific bioassay was performed?) MTT a...
Material diameter measurement,Mean particle size,(What type of material diameter measurement i...,(What type of material diameter measurement i...,(What type of material diameter measurement i...,(What type of material diameter measurement i...,(What type of material diameter measurement i...,Average particle size,Nanoparticle size,(What type of material diameter measurement i...,(What type of material diameter measurement i...,...,(What type of material diameter measurement i...,particle size,(What type of material diameter measurement i...,particle size,(What type of material diameter measurement i...,Particle size,Nanoparticle size,(What type of material diameter measurement i...,Particle size,(What type of material diameter measurement i...
Numerical value,8.2,(What is the numerical value of the diameter ...,(What is the numerical value of the diameter ...,(What is the numerical value of the diameter ...,(What is the numerical value of the diameter ...,(What is the numerical value of the diameter ...,10.2,10,(What is the numerical value of the diameter ...,(What is the numerical value of the diameter ...,...,(What is the numerical value of the diameter ...,10.2,(What is the numerical value of the diameter ...,0.8,(What is the numerical value of the diameter ...,10-20 nm,10,(What is the numerical value of the diameter ...,10,(What is the numerical value of the diameter ...
Units,nm,(What are the units of the diameter measureme...,(What are the units of the diameter measureme...,(What are the units of the diameter measureme...,(What are the units of the diameter measureme...,(What are the units of the diameter measureme...,nm,nm,(What are the units of the diameter measureme...,(What are the units of the diameter measureme...,...,(What are the units of the diameter measureme...,nm,(What are the units of the diameter measureme...,μm,(What are the units of the diameter measureme...,nm,nm,(What are the units of the diameter measureme...,nm,(What are the units of the diameter measureme...
Graphene oxide dose,units mg/mL,units (What are the units of the graphene oxi...,units (What are the units of the graphene oxi...,units (What are the units of the 

## Coating
Extracting adverse outcomes related to diameter from each paper:

In [9]:
coating = """
    Adverse outcome (What was the adverse outcome?):
    Bioassay (What specific bioassay was performed?):
    Coating (What type of nanomaterial coating is reported?):
    Graphene oxide dose (What was the graphene oxide dose that produced this  outcome?):
    Graphene oxide dose units (What are the units of the graphene oxide dose that produced this  outcome?):
    Organism/Biological system (What organism or biological system was the assay performed on (include strain, if applicable)?):"""

In [10]:
results = {}
for document in documents:
    query = f"Select the document with name: {document}"
    docs = docsearch.as_retriever().get_relevant_documents(query)
    chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff",    prompt=doc_prompt)
    
    res = chain({"input_documents": docs, "question": coating},    return_only_outputs=False)
    output = res['output_text'].split("\n")
    results[document] = output
    
output_keys=["Adverse outcome","Bioassay", "Coating type", "Graphene oxide dose", "Graphene oxide dose units", "Organism/Biological system", "score", "Source", "Passage"]
coating = {key:{} for key in results}
for key in results:
    for output in output_keys:
        for value in results[key]:
            if output in value:
                new_val = value.split(output)[1]
                if ':' in new_val:
                    new_val = new_val.replace(":", "")
                    coating[key][output] = new_val
df_coating = pd.DataFrame(coating) # Not needed

df_coating.to_csv('../data/results/graphene-oxide/coating.csv')
df_coating

,j.toxlet.2010.11.016.xml,C2JM31396K.pdf,nn101097v.pdf,j.biomaterials.2013.01.001.xml,acsomega.2c03171.xml,tx400385x.pdf,j.biomaterials.2012.07.040.xml,j.biomaterials.2012.05.064.xml,smll.201201546.pdf,nn202699t.pdf,...,s11671-010-9751-6.xml,j.envpol.2017.12.034.xml,cdd.2010.11.pdf,nl202515a.pdf,smll.201102743.pdf,j.biomaterials.2012.02.021.xml,j.biomaterials.2012.11.001.xml,la203607w.pdf,am300253c.pdf,ijms221910578.xml
Adverse outcome,(What was the adverse outcome?) DNA damage,Apoptosis,Cell viability reduction,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,...",Cell viability reduction,Cell viability reduction,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,...",(What was the adverse outcome?) DNA damage,...,"apoptosis, ROS formation or oxidative stress,...",Apoptosis,Cell viability reduction,Apoptosis,apoptosis,apoptosis,Cell viability reduction,(What was the adverse outcome?) Cell viabilit...,(What was the adverse outcome?) DNA damage,"apoptosis, ROS formation or oxidative stress,..."
Bioassay,(What specific bioassay was performed?) Comet...,Flow cytometry,MTT assay,MTT assay,,MTT assay,MTT assay,MTT assay,,(What specific bioassay was performed?) Comet...,...,,MTT assay,MTT assay,Cell viability reduction,MTT assay,cell viability reduction,MTT assay,(What specific bioassay was performed?) MTT a...,(What specific bioassay was performed?) Comet...,MTT assay
Graphene oxide dose,units (What are the units of the graphene oxi...,units μg/mL,units mg/mL,units mg/mL,units,units mg/mL,units mg/mL,units mg/mL,units,units (What are the units of the graphene oxi...,...,units,units mg/mL,units mg/mL,units mg/L,units mg/mL,units mg/ml,units mg/mL,units (What are the units of the graphene oxi...,units (What are the units of the graphene oxi...,units mg/mL
Graphene oxide dose units,(What are the units of the graphene oxide dos...,μg/mL,mg/mL,mg/mL,,mg/mL,mg/mL,mg/mL,,(What are the units of the graphene oxide dos...,...,,mg/mL,mg/mL,mg/L,mg/mL,mg/ml,mg/mL,(What are the units of the graphene oxide dos...,(What are the units of the graphene oxide dos...,mg/mL
Organism/Biological system,(What organism or biological system was the a...,Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),,Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),,(What organism or biological system was the a...,...,,Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Daphnia magna,Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),(What organism or biological system was the a...,(What organism or biological system was the a...,Human umbilical vein endothelial cells (HUVECs)
Source,"Sánchez-García, M. J., & López-López, M. (201...",42961212012458/MAIN/application/pdf/6a81103c5...,42961213000094/MAIN/application/pdf/803c04541...,10.1016/j.biomaterials.2012.11.001,,"""Graphene oxide induces cytotoxicity in human...",10.1016/j.biomaterials.2013.01.001,10.1016/j.biomaterials.2012.11.001,"1-s2.0-S0142961212012458-main.pdf, 1-s2.0-S01...",1-s2.0-S0142961212012458-main.pdf,...,,S0142961212001871-main.pdf,42961213000094/MAIN/application/pdf/803c04541...,ijms-22-10578-s001.zip,1-s2.0-S0142961212006187-main.pdf,10.1016/j.biomaterials.2012.02.021,10.1016/j.biomaterials.2012.11.001,42961212006187/MAIN/application/pdf/b7769e3aa...,"Lett. 2005, 95, 225507",S026974911733470X-mmc1.xml
Passage,"""The comet assay was used to evaluate DNA dam...","""The cells were exposed to 0.5 μg/mL of GO fo...","""The cell viability was significantly decreas...","""The MTT assay was used to evaluate the cell ...",,"""The results showed that GO induced a dose-de...","""The cell viability of HUVECs was significant...","""The cell viability of

## Dose
Extracting adverse outcomes related to dosage from each paper:

In [11]:
dose = """
    Adverse outcome (What was the adverse outcome?):
    Bioassay (What specific bioassay was performed?):
    Hazardous graphene oxide dose (What was the numerical value of the graphene oxide dose that produced this  outcome?):
    Hazardous graphene oxide dose units (What are the units of the graphene oxide dose that produced this  outcome?):
    Non-hazardous graphene oxide dose (What was the graphene oxide dose that that did not lead to an adverse outcome?):
    Non-hazardous graphene oxide dose units (What are the units of the graphene oxide dose that did not lead to an adverse outcome?):
    Organism/Biological system (What organism or biological system was the assay performed on (include strain, if applicable)?):
    Hazardous diameter (What is the diameter of the graphene oxide that produced this outcome?): 
    Non-hazardous diameter (What is the diameter of the graphene oxide that did not produce an outcome?): """

In [12]:
results = {}
for document in documents:
    query = f"Select the document with name: {document}"
    docs = docsearch.as_retriever().get_relevant_documents(query)
    chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff",    prompt=doc_prompt)
    
    res = chain({"input_documents": docs, "question": dose},    return_only_outputs=False)
    output = res['output_text'].split("\n")
    results[document] = output
    
output_keys=["Adverse outcome","Bioassay", "Hazardous dose", "Hazardous dose units", "Non-hazardous dose", "Non-hazardous dose units", "Organism/Biological system", "Hazardous diameter", "Non-hazardous diameter" "score", "Source", "Passage"]
dose = {key:{} for key in results}
for key in results:
    for output in output_keys:
        for value in results[key]:
            if output in value:
                new_val = value.split(output)[1]
                if ':' in new_val:
                    new_val = new_val.replace(":", "")
                    dose[key][output] = new_val
df_dose = pd.DataFrame(dose) # Not needed

df_dose.to_csv('../data/results/graphene-oxide/dose.csv')
df_dose

,j.toxlet.2010.11.016.xml,C2JM31396K.pdf,nn101097v.pdf,j.biomaterials.2013.01.001.xml,acsomega.2c03171.xml,tx400385x.pdf,j.biomaterials.2012.07.040.xml,j.biomaterials.2012.05.064.xml,smll.201201546.pdf,nn202699t.pdf,...,s11671-010-9751-6.xml,j.envpol.2017.12.034.xml,cdd.2010.11.pdf,nl202515a.pdf,smll.201102743.pdf,j.biomaterials.2012.02.021.xml,j.biomaterials.2012.11.001.xml,la203607w.pdf,am300253c.pdf,ijms221910578.xml
Adverse outcome,Cell viability reduction,Cell viability reduction,Cell viability reduction,Cell viability reduction,apoptosis,DNA damage,Cell viability reduction,Cell viability reduction,apoptosis,DNA damage,...,apoptosis,Cell viability reduction,Cell viability reduction,Apoptosis,Cell viability reduction,apoptosis,Apoptosis,Cell viability reduction,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,..."
Bioassay,MTT assay,MTT assay,MTT assay,MTT assay,cell viability assay,Comet assay,MTT assay,MTT assay,MTT assay,Comet assay,...,MTT assay,MTT assay,MTT assay,MTT assay,MTT assay,MTT assay,MTT assay,MTT assay,MTT assay,MTT assay
Organism/Biological system,Human embryonic kidney cells (HEK293),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human embryonic kidney cells (HEK293),Human lymphocytes,Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human lymphocytes,...,Human embryonic kidney cells (HEK293),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human embryonic kidney cells (HEK293),Human umbilical vein endothelial cells (HUVECs)
Hazardous diameter,2 nm,2-3 nm,2-3 nm,2 nm,2 nm,2-3 nm,2-3 nm,Not applicable,2 nm,2-3 nm,...,2 nm,2 nm,2-3 nm,2 nm,2-3 nm,2 nm,2-3 nm,2-3 nm,2 nm,2 nm
Source,"Sánchez-Sánchez, C., et al. ""Graphene Oxide N...",42961212012458/MAIN/application/pdf/6a81103c5...,42961213000094/MAIN/application/pdf/803c04541...,BIOMATERIALS,mmc2.doc,8356/MAIN/application/pdf/4534f832df349b8189d...,10.1016/j.biomaterials.2012.02.021,"Biomaterials, Volume 33, Issue 33, December 2...",S026974911733470X-main.pdf,1-s2.0-S0142961212012458-main.pdf,...,S0142961212012458,S0142961212006187-mmc1.doc,42961213000094/MAIN/application/pdf/803c04541...,International Journal of Molecular Sciences,1-s2.0-S0142961212006187-main.pdf,10.1016/j.biomaterials.2012.02.021,10.1016/j.biomaterials.2012.11.001,42961212006187/MAIN/application/pdf/b7769e3aa...,"Lett. 2005, 95, 225507",S026974911733470X-mmc1.xml
Passage,"""The highest cytotoxicity was observed at 0.5...","""The results showed that the cell viability d...","""The cell viability of HUVECs was significant...",The MTT assay was used to evaluate the cell v...,"""The cell viability assay showed that 0.5 mg/...","""The comet assay was used to evaluate the DNA...","""The cell viability of HUVECs was significant...","""The cell viability of HUVECs was significant...","""The results showed that the apoptosis rate o...","""The comet assay was used to evaluate the DNA...",...,"""The results showed that the apoptosis rate o...","""The results showed that the cell viability d...","""The MTT assay was used to evaluate the cell ...","""The results showed that the apoptosis rate o...","""The results showed that the cell viability d...","""The MTT assay showed that 0.5 mg/mL GO with ...","""The results showed that the apoptosis rate o...","""The MTT assay was used to evaluate the cell ...","""The cell viability was significantly reduced...","""The results showed that the GO with a diamet..."


## Shape

Extracting adverse outcomes related to shape from each paper

In [13]:
shape = """
    Adverse outcome (What was the adverse outcome?):
    Bioassay (What specific bioassay was performed?):
    Graphene oxide dose (What was the graphene oxide dose that produced this  outcome?):
    Graphene oxide dose units (What are the units of the graphene oxide dose that produced this  outcome?):
    Organism/Biological system (What organism or biological system was the assay performed on (include strain, if applicable)?):
    Hazardous shape (What is the shape of the graphene oxide that produced this outcome?):
    Hazardous diameter (What is the diameter of the graphene oxide that produced this outcome?): 
    Non-hazardous shape (What is the shape of the graphene oxide that did not produce this outcome?):
    Non-hazardous diameter (What is the diameter of the graphene oxide that did not produce this outcome?):"""

In [14]:
results = {}
for document in documents:
    query = f"Select the document with name: {document}"
    docs = docsearch.as_retriever().get_relevant_documents(query)
    chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff",    prompt=doc_prompt)
    
    res = chain({"input_documents": docs, "question": shape},    return_only_outputs=False)
    output = res['output_text'].split("\n")
    results[document] = output
    
output_keys=["Adverse outcome","Bioassay", "Hazardous dose", "Hazardous dose units", "Organism/Biological system", "Hazardous shape", "Hazardous diameter", "Non-hazardous shape", "Non-hazardous diameter" , "score", "Source", "Passage"]
shape = {key:{} for key in results}
for key in results:
    for output in output_keys:
        for value in results[key]:
            if output in value:
                new_val = value.split(output)[1]
                if ':' in new_val:
                    new_val = new_val.replace(":", "")
                    shape[key][output] = new_val
df_shape = pd.DataFrame(shape) # Not needed

df_shape.to_csv('../data/results/graphene-oxide/shape.csv')
df_shape

,j.toxlet.2010.11.016.xml,C2JM31396K.pdf,nn101097v.pdf,j.biomaterials.2013.01.001.xml,acsomega.2c03171.xml,tx400385x.pdf,j.biomaterials.2012.07.040.xml,j.biomaterials.2012.05.064.xml,smll.201201546.pdf,nn202699t.pdf,...,s11671-010-9751-6.xml,j.envpol.2017.12.034.xml,cdd.2010.11.pdf,nl202515a.pdf,smll.201102743.pdf,j.biomaterials.2012.02.021.xml,j.biomaterials.2012.11.001.xml,la203607w.pdf,am300253c.pdf,ijms221910578.xml
Adverse outcome,Apoptosis,Cell viability reduction,Cell viability reduction,Apoptosis,"apoptosis, ROS formation or oxidative stress,...",DNA damage,Apoptosis,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,...",DNA damage,...,apoptosis,Apoptosis,Apoptosis,Apoptosis,Apoptosis,"apoptosis, ROS formation or oxidative stress,...",Apoptosis,Cell viability reduction,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,..."
Bioassay,Flow cytometry,MTT assay,MTT assay,Cell viability assay,,Comet assay,Flow cytometry,MTT assay,,Comet assay,...,MTT assay,Flow cytometry,Flow cytometry,MTT assay,Flow cytometry,,Cell viability assay,MTT assay,MTT assay,MTT assay
Organism/Biological system,Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),,Human lymphocytes,Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),,Human lymphocytes,...,Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),,Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs),Human umbilical vein endothelial cells (HUVECs)
Hazardous shape,Nanosheets,Nanosheets,nanosheets,Nanoplatelets,,nanosheets,Nanoplatelets,nanosheets,,nanosheets,...,nanosheets,Nanoplatelets,Nanosheets,Nanoplatelets,Nanosheets,,Nanoplatelets,Nanosheets,nanosheets,nanosheets
Hazardous diameter,10-20 nm,2-3 nm,10-20 nm,10 nm,,2-3 nm,10-20 nm,10-20 nm,,2-3 nm,...,10-20 nm,10-20 nm,2-3 nm,10 nm,2-3 nm,,10-20 nm,2-3 nm,10-20 nm,10 nm
Non-hazardous shape,Nanorods,Nanorods,nanorods,Nanorods,,nanoribbons,Nanorods,nanorods,,nanoribbons,...,nanorods,Nanorods,Nanorods,Nanorods,Nanorods,,Nanorods,Nanorods,nanorods,nanorods
Non-hazardous diameter,40-50 nm,10-20 nm,40-50 nm,50 nm,,10-20 nm,40-50 nm,50-100 nm,,10-20 nm,...,50-100 nm,40-50 nm,10-20 nm,10 nm,10-20 nm,,50-100 nm,10-20 nm,50-100 nm,50 nm
Source,"Journal of Hazardous Materials, Volume 343, 1...",42961212012458/MAIN/application/pdf/6a81103c5...,"""Graphene oxide nanosheets induce cytotoxicit...",10.1016/j.biomaterials.2013.01.001,,8356/MAIN/application/pdf/4534f832df349b8189d...,"10.1016/j.biomaterials.2012.02.021, 10.1016/j...","Biomaterials, Volume 33, Issue 33, December 2...","1-s2.0-S0142961212012458-main.pdf, 1-s2.0-S01...",1-s2.0-S0142961212012458-main.pdf,...,S0142961212012458,S0142961212006187-mmc1.doc,42961213000094/MAIN/application/pdf/803c04541...,International Journal of Molecular Sciences,1-s2.0-S0142961212006187-main.pdf,10.1016/j.biomaterials.2012.02.021,10.1016/j.biomaterials.2012.11.001,42961212006187/MAIN/application/pdf/b7769e3aa...,Sci.,'Graphene oxide nanosheets induce apoptosis a...
Passage,"""The apoptosis rate of HUVECs was significant...","""The MTT assay showed that GO nanosheets (2–3...","""The results showed that GO nanosheets (10–20...","""The results showed that the graphene oxide n...",,"""The comet assay was used to evaluate the DNA...","""The apoptosis rate of HUVECs was significant...","""The results showed that the nanosheets (10–2...",,"""The comet assay was used to evaluate the DNA...",...,"""The MTT assay showed that the nanosheets (10...","""The results showed that GO nanoplatelets (10...","""The nanosheets (2–3 nm) 

## Time of exposure

Extracting adverse outcomes related to time of exposure from each paper

In [15]:
time = """
    Adverse outcome (What was the adverse outcome?):
    Bioassay (What specific bioassay was performed?):
    Graphene oxide dose (What was the graphene oxide dose that produced this  outcome?):
    Graphene oxide dose units (What are the units of the graphene oxide dose that produced this  outcome?):
    Coating (What type of nanomaterial coating is reported?):
    Organism/Biological system (What organism or biological system was the assay performed on (include strain, if applicable)?):
    Hazardous shape (What is the shape of the graphene oxide that produced this outcome?):
    Hazardous diameter (What is the diameter of the graphene oxide that produced this outcome?): 
    Non-hazardous shape (What is the shape of the graphene oxide that did not produce this outcome?):
    Non-hazardous diameter (What is the diameter of the graphene oxide that did not produce this outcome?):
    Hazardous time point (At which time points did graphene oxide produce the adverse effect?):
    Non-hazardous time point (At which time points did graphene oxide not produce the adverse effect?):"""

In [28]:
results = {}
for document in documents:
    query = f"Select the document with name: {document}"
    docs = docsearch.as_retriever().get_relevant_documents(query)
    chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff",    prompt=doc_prompt)
    
    res = chain({"input_documents": docs, "question": time},    return_only_outputs=False)
    output = res['output_text'].split("\n")
    results[document] = output
    
output_keys=["Adverse outcome","Bioassay", "Hazardous dose", "Hazardous dose units", "Coating", "Organism/Biological system", "Hazardous shape", "Hazardous diameter", "Non-hazardous shape", "Non-hazardous diameter" , "score", "Source", "Passage"]
time = {key:{} for key in results}
for key in results:
    for output in output_keys:
        for value in results[key]:
            if output in value:
                new_val = value.split(output)[1]
                if ':' in new_val:
                    new_val = new_val.replace(":", "")
                    time[key][output] = new_val
df_time = pd.DataFrame(time) # Not needed



Fix outputs

In [21]:
def remove_parentheses(text):
    return re.sub(r'\([^)]*\)', '', text)

In [23]:
df_diameter[df_diameter.columns.difference(['Passage'])] = df_diameter[df_diameter.columns.difference(['Passage'])].applymap(remove_parentheses)
df_diameter.to_csv('../data/results/graphene-oxide/diameter.csv')
df_diameter

,j.toxlet.2010.11.016.xml,C2JM31396K.pdf,nn101097v.pdf,j.biomaterials.2013.01.001.xml,acsomega.2c03171.xml,tx400385x.pdf,j.biomaterials.2012.07.040.xml,j.biomaterials.2012.05.064.xml,smll.201201546.pdf,nn202699t.pdf,...,s11671-010-9751-6.xml,j.envpol.2017.12.034.xml,cdd.2010.11.pdf,nl202515a.pdf,smll.201102743.pdf,j.biomaterials.2012.02.021.xml,j.biomaterials.2012.11.001.xml,la203607w.pdf,am300253c.pdf,ijms221910578.xml
Adverse outcome,Apoptosis,Cell viability reduction,Cell viability reduction,Cell viability reduction,apoptosis,Cell viability reduction,Cell viability reduction,Apoptosis,apoptosis,DNA damage,...,apoptosis,apoptosis,Cell viability reduction,apoptosis,apoptosis,Cell viability reduction,Apoptosis,Cell viability reduction,Cell viability reduction,apoptosis
Bioassay,MTT assay,MTT assay,MTT assay,MTT assay,MTT assay,MTT assay,MTT assay,Flow cytometry,Flow cytometry,Comet assay,...,MTT assay,MTT assay,MTT assay,MTT assay,Flow cytometry,MTT assay,Flow cytometry,MTT assay,Cytotoxicity assay,MTT assay
Material diameter measurement,Mean particle size,Particle size,Particle size,Particle size,particle size,Particle size,Average particle size,Nanoparticle size,Particle size,Tail moment,...,particle size,particle size,Particle size,particle size,Particle size,Particle size,Nanoparticle size,Particle size,Particle size,particle size
Numerical value,8.2,10,10,10,10 nm,10,10.2,10,10,0.9,...,10,10.2,10,0.8,10,10-20 nm,10,10,10,10
Units,nm,nm,nm,nm,nm,nm,nm,nm,nm,μm,...,nm,nm,nm,μm,nm,nm,nm,nm,nm,nm
Graphene oxide dose,units mg/mL,units mg/mL,units mg/mL,units mg/mL,units mg/mL,units mg/mL,units mg/mL,units mg/mL,units mg/mL,units mg/mL,...,units mg/mL,units mg/mL,units mg/mL,units mg/mL,units mg/mL,units mg/ml,units mg/mL,units mg/mL,units mg/mL,units mg/mL
Graphene oxide dose units,mg/mL,mg/mL,mg/mL,mg/mL,mg/mL,mg/mL,mg/mL,mg/mL,mg/mL,mg/mL,...,mg/mL,mg/mL,mg/mL,mg/mL,mg/mL,mg/ml,mg/mL,mg/mL,mg/mL,mg/mL
Organism/Biological system,Human umbilical vein endothelial cells,?) Human embryonic kidney cells,?) Human umbilical vein endothelial cells,?) Human umbilical vein endothelial cells,?) Human umbilical vein endothelial cells,?) Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,?) Human umbilical vein endothelial cells,?) Human embryonic kidney cells,...,?) Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,?) Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,?) Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,?) Human embryonic kidney cells,Human embryonic kidney cells,?) Human umbilical vein endothelial cells
Passage,"""The mean particle size of GO was 8.2 nm .""","""The particle size of GO was 10 nm, as determ...","""The particle size of GO was 10 nm, as determ...","""The particle size of GO was 10 nm, and the d...","""The particle size of GO was 10 nm, as determ...","""The particle size of GO was 10 nm, as determ...","""The average particle size of GO was 10.2 nm ...","""Flow cytometry was used to measure the apopt...","""Flow cytometry analysis showed that 0.5 mg/m...","""The comet assay was used to evaluate the DNA...",...,"""The particle size of GO was 10 nm, as determ...","""The particle size of GO was 10.2 nm . The ce...","""The MTT assay was used to evaluate the cell ...","""The particle size of the GO was 0.8 μm, as d...","""Flow cytometry was used to measure the apopt...","""The particle size of GO was 10–20 nm, as det...","""Flow cytometry was used to measure the apopt...","""The MTT assay was used to evaluate the cell ...","""The cytotoxicity assay was performed on huma...","""The particle size of GO was 10 nm, as determ..."


In [24]:
df_coating[df_coating.columns.difference(['Passage'])] = df_coating[df_coating.columns.difference(['Passage'])].applymap(remove_parentheses)
df_coating.to_csv('../data/results/graphene-oxide/coating.csv')
df_coating

,j.toxlet.2010.11.016.xml,C2JM31396K.pdf,nn101097v.pdf,j.biomaterials.2013.01.001.xml,acsomega.2c03171.xml,tx400385x.pdf,j.biomaterials.2012.07.040.xml,j.biomaterials.2012.05.064.xml,smll.201201546.pdf,nn202699t.pdf,...,s11671-010-9751-6.xml,j.envpol.2017.12.034.xml,cdd.2010.11.pdf,nl202515a.pdf,smll.201102743.pdf,j.biomaterials.2012.02.021.xml,j.biomaterials.2012.11.001.xml,la203607w.pdf,am300253c.pdf,ijms221910578.xml
Adverse outcome,DNA damage,Apoptosis,Cell viability reduction,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,...",Cell viability reduction,Cell viability reduction,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,...",DNA damage,...,"apoptosis, ROS formation or oxidative stress,...",Apoptosis,Cell viability reduction,Apoptosis,apoptosis,apoptosis,Cell viability reduction,Cell viability reduction,DNA damage,"apoptosis, ROS formation or oxidative stress,..."
Bioassay,Comet assay,Flow cytometry,MTT assay,MTT assay,,MTT assay,MTT assay,MTT assay,,Comet assay,...,,MTT assay,MTT assay,Cell viability reduction,MTT assay,cell viability reduction,MTT assay,MTT assay,Comet assay,MTT assay
Graphene oxide dose,units μg/mL,units μg/mL,units mg/mL,units mg/mL,units,units mg/mL,units mg/mL,units mg/mL,units,units mg/mL,...,units,units mg/mL,units mg/mL,units mg/L,units mg/mL,units mg/ml,units mg/mL,units mg/mL,units mg/mL,units mg/mL
Graphene oxide dose units,μg/mL,μg/mL,mg/mL,mg/mL,,mg/mL,mg/mL,mg/mL,,mg/mL,...,,mg/mL,mg/mL,mg/L,mg/mL,mg/ml,mg/mL,mg/mL,mg/mL,mg/mL
Organism/Biological system,?) Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,,?) Human lymphocytes,...,,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Daphnia magna,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,?) Human umbilical vein endothelial cells,?) Human lymphocytes,Human umbilical vein endothelial cells
Source,"Sánchez-García, M. J., & López-López, M. . Cy...",42961212012458/MAIN/application/pdf/6a81103c5...,42961213000094/MAIN/application/pdf/803c04541...,10.1016/j.biomaterials.2012.11.001,,"""Graphene oxide induces cytotoxicity in human...",10.1016/j.biomaterials.2013.01.001,10.1016/j.biomaterials.2012.11.001,"1-s2.0-S0142961212012458-main.pdf, 1-s2.0-S01...",1-s2.0-S0142961212012458-main.pdf,...,,S0142961212001871-main.pdf,42961213000094/MAIN/application/pdf/803c04541...,ijms-22-10578-s001.zip,1-s2.0-S0142961212006187-main.pdf,10.1016/j.biomaterials.2012.02.021,10.1016/j.biomaterials.2012.11.001,42961212006187/MAIN/application/pdf/b7769e3aa...,"Lett. 2005, 95, 225507",S026974911733470X-mmc1.xml
Passage,"""The comet assay was used to evaluate DNA dam...","""The cells were exposed to 0.5 μg/mL of GO fo...","""The cell viability was significantly decreas...","""The MTT assay was used to evaluate the cell ...",,"""The results showed that GO induced a dose-de...","""The cell viability of HUVECs was significant...","""The cell viability of HUVECs was significant...",,"""The comet assay was used to evaluate the DNA...",...,,"""The cells were exposed to 0.5 mg/mL of GO fo...","""The cell viability was significantly decreas...","""The results showed that the cell viability o...","""The results showed that the apoptosis rate o...","""The results showed that 0.5 mg/ml GO induced...","""The cell viability of HUVECs was significant...","""The cells were exposed to 0.5 mg/mL of GO fo...","""The comet assay was used to evaluate the gen...","""The cells were exposed to 0.5 mg/mL graphene..."


In [25]:
df_dose[df_dose.columns.difference(['Passage'])] = df_dose[df_dose.columns.difference(['Passage'])].applymap(remove_parentheses)
df_dose.to_csv('../data/results/graphene-oxide/dose.csv')
df_dose

,j.toxlet.2010.11.016.xml,C2JM31396K.pdf,nn101097v.pdf,j.biomaterials.2013.01.001.xml,acsomega.2c03171.xml,tx400385x.pdf,j.biomaterials.2012.07.040.xml,j.biomaterials.2012.05.064.xml,smll.201201546.pdf,nn202699t.pdf,...,s11671-010-9751-6.xml,j.envpol.2017.12.034.xml,cdd.2010.11.pdf,nl202515a.pdf,smll.201102743.pdf,j.biomaterials.2012.02.021.xml,j.biomaterials.2012.11.001.xml,la203607w.pdf,am300253c.pdf,ijms221910578.xml
Adverse outcome,Cytotoxicity,Cytotoxicity,Cytotoxicity,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,...",DNA damage,Cell viability reduction,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,...",DNA damage,...,apoptosis,Cell viability reduction,Cytotoxicity,Apoptosis,Apoptosis,"apoptosis, ROS formation or oxidative stress,...",Cell viability reduction,Cell viability reduction,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,..."
Bioassay,MTT assay,MTT assay,MTT assay,MTT assay,,Comet assay,MTT assay,MTT assay,,Comet assay,...,MTT assay,MTT assay,MTT assay,MTT assay,Flow cytometry,cytotoxicity assay,MTT assay,MTT assay,MTT assay,MTT assay
Coating,Graphene oxide,Graphene oxide,Graphene oxide,Graphene oxide,,Graphene oxide,Graphene oxide,Graphene oxide,,Graphene oxide,...,graphene oxide,Graphene oxide,Graphene oxide,Graphene oxide,Graphene oxide,graphene oxide,Graphene oxide,Graphene oxide,Graphene oxide,graphene oxide
Organism/Biological system,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,,Human umbilical vein endothelial cells,...,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells
Hazardous shape,Nanosheets,Nanosheets,Nanosheets,Nanosheets,,Nanosheets,Nanosheets,Nanosheets,,Nanosheets,...,nanosheets,Nanosheets,Nanosheets,Nanosheets,Nanosheets,nanosheets,Nanosheets,Nanosheets,Nanosheets,nanosheets
Hazardous diameter,2-3 nm,2-3 nm,2-3 nm,2-3 nm,,2-3 nm,2-3 nm,2-3 nm,,2-3 nm,...,2-3 nm,2-3 nm,2-3 nm,2-3 nm,2-3 nm,2-3 nm,2-3 nm,2-3 nm,2-3 nm,2-3 nm
Non-hazardous shape,Nanoparticles,Nanoparticles,Nanoparticles,Nanoparticles,,Nanoparticles,Nanoparticles,Nanorods,,Nanoparticles,...,nanorods,Nanorods,Nanoparticles,Nanoparticles,Nanorods,nanorods,Nanoparticles,Nanoparticles,Nanoparticles,nanoribbons
Non-hazardous diameter,10-20 nm,10-20 nm,10-20 nm,10-20 nm,,10-20 nm,10-20 nm,10-20 nm,,10-20 nm,...,10-20 nm,10-20 nm,10-20 nm,10-20 nm,10-20 nm,10-20 nm,10-20 nm,10-20 nm,10-20 nm,10-20 nm
Source,'Cytotoxicity of graphene oxide nanosheets an...,42961212012458/MAIN/application/pdf/6a81103c5...,42961213000094/MAIN/application/pdf/803c04541...,10.1016/j.biomaterials.2013.01.001,,8356/MAIN/application/pdf/4534f832df349b8189d...,10.1016/j.biomaterials.2012.11.001,"Biomaterials, Volume 33, Issue 33, December 2...",,1-s2.0-S0142961212012458-main.pdf,...,S0142961212012458,S0142961212001871-main.pdf,42961213000094/MAIN/application/pdf/803c04541...,International Journal of Molecular Sciences,1-s2.0-S0142961212006187-main.pdf,10.1016/j.biomaterials.2012.02.021,10.1016/j.biomaterials.2012.11.001,42961212006187/MAIN/application/pdf/b7769e3aa...,"Lett. 2005, 95, 225507",'Graphene oxide nanosheets and nanoribbons in...
Passage,'The MTT assay was used to evaluate the cytot...,"""The cytotoxicity of GO nanosheets was highe...","""The cytotoxicity of GO nanosheets was highe...","""The MTT assay showed that the cell viability...",,"""The comet assay was used to evaluate the DNA...","""The MTT assay showed that GO nan

In [26]:
df_shape[df_shape.columns.difference(['Passage'])] = df_shape[df_shape.columns.difference(['Passage'])].applymap(remove_parentheses)
df_shape.to_csv('../data/results/graphene-oxide/shape.csv')
df_shape

,j.toxlet.2010.11.016.xml,C2JM31396K.pdf,nn101097v.pdf,j.biomaterials.2013.01.001.xml,acsomega.2c03171.xml,tx400385x.pdf,j.biomaterials.2012.07.040.xml,j.biomaterials.2012.05.064.xml,smll.201201546.pdf,nn202699t.pdf,...,s11671-010-9751-6.xml,j.envpol.2017.12.034.xml,cdd.2010.11.pdf,nl202515a.pdf,smll.201102743.pdf,j.biomaterials.2012.02.021.xml,j.biomaterials.2012.11.001.xml,la203607w.pdf,am300253c.pdf,ijms221910578.xml
Adverse outcome,Apoptosis,Cell viability reduction,Cell viability reduction,Apoptosis,"apoptosis, ROS formation or oxidative stress,...",DNA damage,Apoptosis,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,...",DNA damage,...,apoptosis,Apoptosis,Apoptosis,Apoptosis,Apoptosis,"apoptosis, ROS formation or oxidative stress,...",Apoptosis,Cell viability reduction,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,..."
Bioassay,Flow cytometry,MTT assay,MTT assay,Cell viability assay,,Comet assay,Flow cytometry,MTT assay,,Comet assay,...,MTT assay,Flow cytometry,Flow cytometry,MTT assay,Flow cytometry,,Cell viability assay,MTT assay,MTT assay,MTT assay
Organism/Biological system,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,,Human lymphocytes,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,,Human lymphocytes,...,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells
Hazardous shape,Nanosheets,Nanosheets,nanosheets,Nanoplatelets,,nanosheets,Nanoplatelets,nanosheets,,nanosheets,...,nanosheets,Nanoplatelets,Nanosheets,Nanoplatelets,Nanosheets,,Nanoplatelets,Nanosheets,nanosheets,nanosheets
Hazardous diameter,10-20 nm,2-3 nm,10-20 nm,10 nm,,2-3 nm,10-20 nm,10-20 nm,,2-3 nm,...,10-20 nm,10-20 nm,2-3 nm,10 nm,2-3 nm,,10-20 nm,2-3 nm,10-20 nm,10 nm
Non-hazardous shape,Nanorods,Nanorods,nanorods,Nanorods,,nanoribbons,Nanorods,nanorods,,nanoribbons,...,nanorods,Nanorods,Nanorods,Nanorods,Nanorods,,Nanorods,Nanorods,nanorods,nanorods
Non-hazardous diameter,40-50 nm,10-20 nm,40-50 nm,50 nm,,10-20 nm,40-50 nm,50-100 nm,,10-20 nm,...,50-100 nm,40-50 nm,10-20 nm,10 nm,10-20 nm,,50-100 nm,10-20 nm,50-100 nm,50 nm
Source,"Journal of Hazardous Materials, Volume 343, 1...",42961212012458/MAIN/application/pdf/6a81103c5...,"""Graphene oxide nanosheets induce cytotoxicit...",10.1016/j.biomaterials.2013.01.001,,8356/MAIN/application/pdf/4534f832df349b8189d...,"10.1016/j.biomaterials.2012.02.021, 10.1016/j...","Biomaterials, Volume 33, Issue 33, December 2...","1-s2.0-S0142961212012458-main.pdf, 1-s2.0-S01...",1-s2.0-S0142961212012458-main.pdf,...,S0142961212012458,S0142961212006187-mmc1.doc,42961213000094/MAIN/application/pdf/803c04541...,International Journal of Molecular Sciences,1-s2.0-S0142961212006187-main.pdf,10.1016/j.biomaterials.2012.02.021,10.1016/j.biomaterials.2012.11.001,42961212006187/MAIN/application/pdf/b7769e3aa...,Sci.,'Graphene oxide nanosheets induce apoptosis a...
Passage,"""The apoptosis rate of HUVECs was significant...","""The MTT assay showed that GO nanosheets sig...","""The results showed that GO nanosheets induc...","""The results showed that the graphene oxide n...",,"""The comet assay was used to evaluate the DNA...","""The apoptosis rate of HUVECs was significant...","""The results showed that the nanosheets caus...",,"""The comet assay was used to evaluate the DNA...",...,"""The MTT assay showed that the nanosheets in...","""The results showed that GO nanoplatelets in...","""The nanosheets induced apoptosis in HUVECs ...","""The MTT assay was used to evaluate the cytot...","""The results showed that the nanosheets indu...",,"""The resul

In [29]:
df_time[df_time.columns.difference(['Passage'])] = df_time[df_time.columns.difference(['Passage'])].applymap(remove_parentheses)
df_time.to_csv('../data/results/graphene-oxide/time.csv')
df_time

,j.toxlet.2010.11.016.xml,C2JM31396K.pdf,nn101097v.pdf,j.biomaterials.2013.01.001.xml,acsomega.2c03171.xml,tx400385x.pdf,j.biomaterials.2012.07.040.xml,j.biomaterials.2012.05.064.xml,smll.201201546.pdf,nn202699t.pdf,...,s11671-010-9751-6.xml,j.envpol.2017.12.034.xml,cdd.2010.11.pdf,nl202515a.pdf,smll.201102743.pdf,j.biomaterials.2012.02.021.xml,j.biomaterials.2012.11.001.xml,la203607w.pdf,am300253c.pdf,ijms221910578.xml
Adverse outcome,Cytotoxicity,Cell viability reduction,Cytotoxicity,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,...",DNA damage,Cell viability reduction,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,...",DNA damage,...,apoptosis,"Apoptosis, ROS formation or oxidative stress,...",Cytotoxicity,Apoptosis,Apoptosis,"apoptosis, ROS formation or oxidative stress,...",Cell viability reduction,Cell viability reduction,Cell viability reduction,"apoptosis, ROS formation or oxidative stress,..."
Bioassay,MTT assay,MTT assay,MTT assay,MTT assay,,Comet assay,MTT assay,MTT assay,,Comet assay,...,MTT assay,MTT assay,MTT assay,MTT assay,Flow cytometry,cytotoxicity assay,MTT assay,MTT assay,MTT assay,MTT assay
Coating,Graphene oxide,Graphene oxide,Graphene oxide,Graphene oxide,,Graphene oxide,Graphene oxide,Graphene oxide,,Graphene oxide,...,graphene oxide,Graphene oxide,Graphene oxide,Graphene oxide,Graphene oxide,graphene oxide,Graphene oxide,Graphene oxide,Graphene oxide,graphene oxide
Organism/Biological system,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,,Human umbilical vein endothelial cells,...,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells,Human umbilical vein endothelial cells
Hazardous shape,Nanosheets,Nanosheets,Nanosheets,Nanosheets,,Nanosheets,Nanosheets,Nanosheets,,Nanosheets,...,nanosheets,Nanosheets,Nanosheets,Nanosheets,Nanosheets,nanosheets,Nanosheets,Nanosheets,Nanosheets,nanosheets
Hazardous diameter,2-3 nm,2-3 nm,2-3 nm,2-3 nm,,2-3 nm,2-3 nm,2-3 nm,,2-3 nm,...,2-3 nm,2-3 nm,2-3 nm,2-3 nm,2-3 nm,2-3 nm,2-3 nm,2-3 nm,2-3 nm,2-3 nm
Non-hazardous shape,Nanoparticles,Nanoparticles,Nanoparticles,Nanoparticles,,Nanoparticles,Nanoparticles,Nanorods,,Nanoparticles,...,nanorods,Nanoparticles,Nanoparticles,Nanoparticles,Nanorods,nanorods,Nanoparticles,Nanoparticles,Nanoparticles,nanoribbons
Non-hazardous diameter,10-20 nm,10-20 nm,10-20 nm,10-20 nm,,10-20 nm,10-20 nm,10-20 nm,,10-20 nm,...,10-20 nm,10-20 nm,10-20 nm,10-20 nm,10-20 nm,10-20 nm,10-20 nm,10-20 nm,10-20 nm,10-20 nm
Source,'Cytotoxicity of graphene oxide nanosheets an...,42961212012458/MAIN/application/pdf/6a81103c5...,42961213000094/MAIN/application/pdf/803c04541...,10.1016/j.biomaterials.2013.01.001,,8356/MAIN/application/pdf/4534f832df349b8189d...,10.1016/j.biomaterials.2012.11.001,"Biomaterials, Volume 33, Issue 33, December 2...",,42961213000094/MAIN/application/pdf/803c04541...,...,S0142961212012458,S0142961212001871-main.pdf,42961213000094/MAIN/application/pdf/803c04541...,International Journal of Molecular Sciences,1-s2.0-S0142961212006187-main.pdf,10.1016/j.biomaterials.2012.02.021,10.1016/j.biomaterials.2012.11.001,42961212006187/MAIN/application/pdf/b7769e3aa...,"Lett. 2005, 95, 225507",'Graphene oxide nanosheets and nanoribbons in...
Passage,'The MTT assay was used to evaluate the cytot...,"""The results showed that GO nanosheets signi...","""The cytotoxicity of GO nanosheets was highe...","""The MTT assay showed that the cell viability...",,"""The comet assay was used t